# AutoGen Osnovni Primjer

U ovom primjeru koda koristit ćete [AutoGen](https://aka.ms/ai-agents/autogen) AI Framework za kreiranje osnovnog agenta.

Cilj ovog primjera je pokazati korake koje ćemo kasnije koristiti u dodatnim primjerima koda prilikom implementacije različitih agentnih obrazaca.


## Uvoz potrebnih Python paketa


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Kreirajte klijenta

U ovom primjeru koristit ćemo [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) za pristup LLM-u.

`model` je definiran kao `gpt-4o-mini`. Pokušajte promijeniti model na neki drugi dostupan na GitHub Models tržištu kako biste vidjeli različite rezultate.

Kao brzi test, jednostavno ćemo pokrenuti jednostavan upit - `Koji je glavni grad Francuske`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definiranje Agenta

Sada kada smo postavili `client` i potvrdili da radi, kreirajmo `AssistantAgent`. Svakom agentu može se dodijeliti:  
**name** - Kratko ime koje će biti korisno za referenciranje u tokovima s više agenata.  
**model_client** - Klijent koji ste kreirali u prethodnom koraku.  
**tools** - Dostupni alati koje agent može koristiti za izvršavanje zadatka.  
**system_message** - Metaprompt koji definira zadatak, ponašanje i ton LLM-a.  

Možete promijeniti sistemsku poruku kako biste vidjeli kako LLM reagira. `tools` ćemo obraditi u Lekciji #4.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Pokreni Agenta

Donja funkcija pokreće agenta. Koristimo metodu `on_message` kako bismo ažurirali stanje Agenta s novom porukom.

U ovom slučaju, ažuriramo stanje s novom porukom korisnika koja glasi: `"Planiraj mi sjajan sunčani odmor"`.

Možete promijeniti sadržaj poruke kako biste vidjeli kako LLM različito reagira.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane ljudskog prevoditelja. Ne preuzimamo odgovornost za bilo kakve nesporazume ili pogrešne interpretacije koje proizlaze iz korištenja ovog prijevoda.
